In [21]:
import polars as pl
import matplotlib.pyplot as plt
import os
import pandas as pd

In [22]:
train_path='/home/ming/GraduateProject/Data/MINDsmall_train'
valid_path='/home/ming/GraduateProject/Data/MINDsmall_dev'

In [23]:
train=pl.read_csv(os.path.join(train_path,"behaviors.txt"),
        separator='\t',
        has_header=False,
        new_columns=[
            'id',
            'user_id',
            'time',
            'history',
            'impressions'
        ])
valid=pl.read_csv(os.path.join(valid_path,"behaviors.txt"),
        separator='\t',
        has_header=False,
        new_columns=[
            'id',
            'user_id',
            'time',
            'history',
            'impressions'
        ])

In [24]:
train = train.with_columns(
    pl.col("time").str.strptime(
        pl.Datetime,
        format="%m/%d/%Y %I:%M:%S %p",
        strict=False
    )
)

valid = valid.with_columns(
    pl.col("time").str.strptime(
        pl.Datetime,
        format="%m/%d/%Y %I:%M:%S %p",
        strict=False
    )
)


In [25]:
valid=valid.with_columns(pl.col('history').fill_null("")
    .str.split(" ")
    .alias("history"))

In [26]:
valid=valid.with_columns(pl.col('impressions').fill_null("")
    .str.split(" ")
    .alias("impressions"))

In [27]:
valid.head()

id,user_id,time,history,impressions
i64,str,datetime[μs],list[str],list[str]
1,"""U80234""",2019-11-15 12:37:50,"[""N55189"", ""N46039"", … ""N35671""]","[""N28682-0"", ""N48740-0"", … ""N29862-0""]"
2,"""U60458""",2019-11-15 07:11:50,"[""N58715"", ""N32109"", … ""N6778""]","[""N20036-0"", ""N23513-1"", … ""N31958-0""]"
3,"""U44190""",2019-11-15 09:55:12,"[""N56253"", ""N1150"", … ""N3259""]","[""N36779-0"", ""N62365-0"", … ""N55237-0""]"
4,"""U87380""",2019-11-15 15:12:46,"[""N63554"", ""N49153"", … ""N28247""]","[""N6950-0"", ""N60215-0"", … ""N4733-0""]"
5,"""U9444""",2019-11-15 08:25:46,"[""N51692"", ""N18285"", … ""N55556""]","[""N5940-1"", ""N23513-0"", … ""N20036-0""]"


In [28]:
train.shape[0]+valid.shape[0]

230117

In [29]:
train['user_id'].unique().count()

50000

In [30]:
valid['user_id'].unique().count()

50000

In [31]:
train['time'].min(),train['time'].max()

(datetime.datetime(2019, 11, 9, 0, 0, 19),
 datetime.datetime(2019, 11, 14, 23, 59, 13))

In [32]:
valid['time'].min(),valid['time'].max()

(datetime.datetime(2019, 11, 15, 0, 0, 1),
 datetime.datetime(2019, 11, 15, 23, 58, 3))

In [33]:
train.filter(pl.col('user_id')=="U239687"),valid.filter(pl.col('user_id')=="U239687")

(shape: (0, 5)
 ┌─────┬─────────┬──────────────┬─────────┬─────────────┐
 │ id  ┆ user_id ┆ time         ┆ history ┆ impressions │
 │ --- ┆ ---     ┆ ---          ┆ ---     ┆ ---         │
 │ i64 ┆ str     ┆ datetime[μs] ┆ str     ┆ str         │
 ╞═════╪═════════╪══════════════╪═════════╪═════════════╡
 └─────┴─────────┴──────────────┴─────────┴─────────────┘,
 shape: (0, 5)
 ┌─────┬─────────┬──────────────┬───────────┬─────────────┐
 │ id  ┆ user_id ┆ time         ┆ history   ┆ impressions │
 │ --- ┆ ---     ┆ ---          ┆ ---       ┆ ---         │
 │ i64 ┆ str     ┆ datetime[μs] ┆ list[str] ┆ list[str]   │
 ╞═════╪═════════╪══════════════╪═══════════╪═════════════╡
 └─────┴─────────┴──────────────┴───────────┴─────────────┘)

In [34]:
train.head()

id,user_id,time,history,impressions
i64,str,datetime[μs],str,str
1,"""U13740""",2019-11-11 09:05:58,"""N55189 N42782 N34694 N45794 N1…","""N55689-1 N35729-0"""
2,"""U91836""",2019-11-12 18:11:30,"""N31739 N6072 N63045 N23979 N35…","""N20678-0 N39317-0 N58114-0 N20…"
3,"""U73700""",2019-11-14 07:01:48,"""N10732 N25792 N7563 N21087 N41…","""N50014-0 N23877-0 N35389-0 N49…"
4,"""U34670""",2019-11-11 05:28:05,"""N45729 N2203 N871 N53880 N4137…","""N35729-0 N33632-0 N49685-1 N27…"
5,"""U8125""",2019-11-12 16:11:21,"""N10078 N56514 N14904 N33740""","""N39985-0 N36050-0 N16096-0 N84…"


In [35]:
train=train.with_columns(pl.col('history').fill_null("")
    .str.split(" ")
    .alias("history"))

In [36]:
train=train.with_columns(pl.col('impressions').fill_null("")
    .str.split(" ")
    .alias("impressions"))

In [37]:
train.write_parquet(os.path.join(train_path,"behaviors.parquet"))
valid.write_parquet(os.path.join(valid_path,"behaviors.parquet"))